In [1]:
import pandas as pd

data = pd.read_csv('data/train.csv', index_col = 0, dtype={'multiple_full_time_jobs': 'boolean', 'combined_multiple_jobs': 'boolean'}).reset_index(drop=True)
strCols = data.select_dtypes(object).columns
for i in strCols:
    data[i] = data[i].str.strip()
#data = data.sample(frac=0.1)
data = data.fillna(True)
data.HIREDT = data.HIREDT.apply(pd.to_datetime, format = "%m/%d/%y")
data['daysexperience'] = (pd.to_datetime('10/7/23', format = "%m/%d/%y") - data.HIREDT ).dt.days
data_num = ['RATE', 'HRSWKD', 'daysexperience']
data_drop = ['AGY', 'HIREDT']#, 'multiple_full_time_jobs', 'combined_multiple_jobs']
data.drop(data_drop, axis=1, inplace=True)

important_features = ['daysexperience', 'RATE', 'RACE', 'SEX', 'MI', 'HRSWKD', 'JC.TITLE']
data_to_onehot = ['NAME', 'MI', 'JOBCLASS', 'JC.TITLE', 'RACE', 'SEX', 'EMPTYPE', 'STATENUM', 'multiple_full_time_jobs', 'combined_multiple_jobs']

In [2]:
from sklearn.model_selection import train_test_split
X = data.drop('ANNUAL', axis=1)
y = data.ANNUAL

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3)

In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import scipy as sp
#regressor = LinearRegression(n_jobs=-1) np.log10 sp.special.exp10
regressor = TransformedTargetRegressor(
        regressor=Ridge(alpha=1e-10), func=None, inverse_func=None
    )
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'infrequent_if_exist', drop='if_binary'), data_to_onehot),
    remainder = 'passthrough',
)
pipe = Pipeline([
    ('transformer', transformer)
    #,('stdScaler', StandardScaler(with_mean=False))
    #,('regressor', regressor)
    ,('regressor', LassoCV(cv=5, random_state=0))
])

In [ ]:
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)